## 4.2 Stop right now, thank you very much

#### Objetivo: 
implemente uma estratégia de Parada Antecipada (Early Stopping) no
processo de treino da rede neural feita em Python puro ou no processo de treino da rede
neural feita em PyTorch

## Introdução

O processo de treinamento de uma rede neural costuma envolver a otimização iterativa dos pesos até um número máximo de épocas seja alcançado. No entanto, treinar por muitas épocas pode levar a **overfitting**, que é quando o modelo começa a ajustar-se demais aos ruídos dos dados de treinamento e perde capacidade de generalização nos exemplos novos. Uma estratégia simples e eficiente para contornar esse problema é a **Parada Antecipada (Early Stopping)**, que interrompe o treino tão logo a performance em um conjunto de validação deixe de melhorar.

Neste notebook implementamos o Early Stopping em uma rede neural feita no Pytorch, aproveitando o mecanismo de `torch.nn` e `torch.optim`.  
Para esse trabalho, vamos usar um dataset associado a dados cardíacos onde será feito um tratamento dos dados(retirada de colunas que não são funcionais, normalização dos dados e divisão em dados de treino, validação e teste). Além disso, será necessários converter os dados em Tensores, que é uma necessidade para usar o Pytotch.
Vamos monitorar a **loss** no conjunto de validação a cada época, salvar o melhor estado de parâmetros e interromper o treino quando não houver melhora após um número fixo de épocas (paciencia), garantindo que não ocorra overfitting.


In [22]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


In [23]:
class MLP(nn.Module):
    def __init__(self, num_dados_entrada, neuronios_c1, neuronios_c2, num_targets):
        super().__init__()
        
        self.camadas = nn.Sequential(
            nn.Linear(num_dados_entrada, neuronios_c1),
            nn.Sigmoid(),
            nn.Linear(neuronios_c1, neuronios_c2),
            nn.Sigmoid(),
            nn.Linear(neuronios_c2, num_targets),
        )
        
    def forward(self, x):
        x = self.camadas(x)
        return x

In [24]:

df = pd.read_csv("C:/Users/marco24038/OneDrive - ILUM ESCOLA DE CIÊNCIA/Redes Neurais e Algoritmos Genéticos/Redes_Neurais_Trabalhos/Feras formidáveis/Fera 4.2/Medicaldataset.csv")
df


,Age,Gender,Heart rate,Systolic blood pressure,Diastolic blood pressure,Blood sugar,CK-MB,Troponin,Result
0,64,1,66,160,83,160.0,1.80,0.012,negative
1,21,1,94,98,46,296.0,6.75,1.060,positive
2,55,1,64,160,77,270.0,1.99,0.003,negative
3,64,1,70,120,55,270.0,13.87,0.122,positive
4,55,1,64,112,65,300.0,1.08,0.003,negative
...,...,...,...,...,...,...,...,...,...
1314,44,1,94,122,67,204.0,1.63,0.006,negative
1315,66,1,84,125,55,149.0,1.33,0.172,positive
1316,45,1,85,168,104,96.0,1.24,4.250,positive
1317,54,1,58,117,68,443.0,5.80,0.359,positive


In [25]:
X = df.drop(columns=["Blood sugar", "Result", "Gender"])
y = df["Blood sugar"].values.reshape(-1, 1)

X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.1, random_state=42)

In [26]:
scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_train = scaler_X.fit_transform(X_train)
X_val = scaler_X.transform(X_val)
X_test = scaler_X.transform(X_test)

y_train = scaler_y.fit_transform(y_train)
y_val = scaler_y.transform(y_val)
y_test = scaler_y.transform(y_test)
y_test

array([[-4.29050602e-02],
       [-7.37248828e-01],
       [-4.92186322e-01],
       [-1.92665481e-01],
       [-6.41946742e-01],
       [ 1.88542862e-01],
       [-6.41946742e-01],
       [-6.41946742e-01],
       [-7.78092579e-01],
       [-8.87009248e-01],
       [ 3.74194920e+00],
       [-5.19415489e-01],
       [ 8.69272046e-01],
       [ 1.07349080e+00],
       [-5.19415489e-01],
       [-4.92186322e-01],
       [-5.60259240e-01],
       [-6.28332158e-01],
       [ 5.36208466e+00],
       [ 2.56615781e-01],
       [-4.10498820e-01],
       [-5.33030073e-01],
       [ 3.33351169e+00],
       [ 6.78667875e-01],
       [-6.41946742e-01],
       [-1.38207146e-01],
       [-7.50863412e-01],
       [-5.87488407e-01],
       [-6.41946742e-01],
       [-7.50863412e-01],
       [-6.14717575e-01],
       [-5.19415489e-01],
       [-6.82790493e-01],
       [-4.10498820e-01],
       [-2.06130916e-03],
       [ 3.79147034e-01],
       [-4.37727987e-01],
       [-6.96405077e-01],
       [-7.2

In [27]:

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)

y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)


In [28]:
NUM_DADOS_DE_ENTRADA = X_train.shape[1] 
NUM_DADOS_DE_SAIDA = 1  
NEURONIOS_C1 = 5  
NEURONIOS_C2 = 3  

minha_mlp = MLP(
    NUM_DADOS_DE_ENTRADA, NEURONIOS_C1, NEURONIOS_C2, NUM_DADOS_DE_SAIDA
)

In [29]:
TAXA_DE_APRENDIZADO = 0.001
otimizador = optim.SGD(minha_mlp.parameters(), lr=TAXA_DE_APRENDIZADO)


fn_perda = nn.MSELoss()

In [30]:
NUM_EPOCAS = 1000
paciencia = 10
min_delta = 0.001
melhor_loss = float("inf")
contador_paciencia = 0

minha_mlp.train()

for epoca in range(NUM_EPOCAS):
    y_pred = minha_mlp(X_train_tensor)

    otimizador.zero_grad()

    loss = fn_perda(y_pred, y_train_tensor)
    
    y_pred_val = minha_mlp(X_val_tensor)
    loss_val = fn_perda(y_pred_val, y_val_tensor)

    loss.backward()

    otimizador.step()
    
    print(f"Época {epoca}: perda treino = {loss.item():.4f}, perda validação = {loss_val.item():.4f}")

    # === EARLY STOPPING COM BASE NA VALIDAÇÃO ===
    if loss_val.item() < melhor_loss - min_delta:
        melhor_loss = loss_val.item()
        contador_paciencia = 0
    else:
        contador_paciencia += 1
        if contador_paciencia >= paciencia:
            print(f"Parada antecipada na época {epoca} com perda da validação = {loss_val.item():.4f}")
            break

Época 0: perda treino = 1.3661, perda validação = 1.3839
Época 1: perda treino = 1.3638, perda validação = 1.3819
Época 2: perda treino = 1.3614, perda validação = 1.3799
Época 3: perda treino = 1.3591, perda validação = 1.3780
Época 4: perda treino = 1.3568, perda validação = 1.3760
Época 5: perda treino = 1.3545, perda validação = 1.3741
Época 6: perda treino = 1.3523, perda validação = 1.3722
Época 7: perda treino = 1.3500, perda validação = 1.3703
Época 8: perda treino = 1.3478, perda validação = 1.3684
Época 9: perda treino = 1.3456, perda validação = 1.3665
Época 10: perda treino = 1.3433, perda validação = 1.3647
Época 11: perda treino = 1.3411, perda validação = 1.3629
Época 12: perda treino = 1.3390, perda validação = 1.3610
Época 13: perda treino = 1.3368, perda validação = 1.3592
Época 14: perda treino = 1.3346, perda validação = 1.3574
Época 15: perda treino = 1.3325, perda validação = 1.3556
Época 16: perda treino = 1.3304, perda validação = 1.3538
Época 17: perda treino =

Época 257: perda treino = 1.0718, perda validação = 1.1533
Época 258: perda treino = 1.0713, perda validação = 1.1530
Época 259: perda treino = 1.0709, perda validação = 1.1527
Época 260: perda treino = 1.0704, perda validação = 1.1524
Época 261: perda treino = 1.0700, perda validação = 1.1522
Época 262: perda treino = 1.0695, perda validação = 1.1519
Época 263: perda treino = 1.0691, perda validação = 1.1516
Época 264: perda treino = 1.0687, perda validação = 1.1513
Época 265: perda treino = 1.0682, perda validação = 1.1510
Época 266: perda treino = 1.0678, perda validação = 1.1508
Época 267: perda treino = 1.0674, perda validação = 1.1505
Época 268: perda treino = 1.0670, perda validação = 1.1502
Época 269: perda treino = 1.0666, perda validação = 1.1500
Época 270: perda treino = 1.0661, perda validação = 1.1497
Época 271: perda treino = 1.0657, perda validação = 1.1494
Época 272: perda treino = 1.0653, perda validação = 1.1492
Época 273: perda treino = 1.0649, perda validação = 1.14

## Conclusão


Em síntese, este notebook demonstrou como o early stopping pode atuar como um critério de parada inteligente em redes neurais, interrompendo o processo assim que a métrica de validação deixa de melhorar. Observamos, que em geral há um ponto em que o modelo começa a dar overfitting no conjunto de treino e a performance em validação estagna ou piora; ao aplicar early stopping, capturamos exatamente a iteração ótima, garantindo maior capacidade do modelo.

Além do ganho em qualidade do modelo, o uso desse mecanismo também trouxe economia de tempo de treinamento, evitando ciclos extras de ajuste que não trazem benefício de validação. Como resultado, conseguimos um modelo com menor custo computacional e reduzimos o risco de overfitting sem precisar adivinhar o número exato de épocas. 

## Referências



- Cassar, Daniel R. - Material da disciplina de Redes Neurais e Algoritmos genéticos. 2025
- [medium](https://cyborgcodes.medium.com/what-is-early-stopping-in-deep-learning-eeb1e710a3cf)
- [dataset](https://www.kaggle.com/)